In [2]:
import sys
import pandas as pd
sys.path.insert(0, '/home/jmunson-mcgee/')
from JMM_functions import *

pd.set_option("display.max_rows", 6)

outdir='/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/split_files'
safe_make_dir(outdir)
os.chdir(outdir)

Folder already exists


## Create folders and txt files for NCBI submission

### This notebook needs to have three things
1. a biosample attributes txt file that has all of the biosample data for every SAG that is going to be submitted to NCBI 
1. a genome attributes txt file that has assembly data for every sample 
    - This file must be in the same order as the biosample_attributes file above to ensure that each "chunk" contains the same SAGs
1. A single folder with all of the SAG fasta files that will be uploaded

### This notebook will create 
1. biosample_attribute chunk files with 400 SAGs each
1. Genome_attribute chunk files with 400 SAGs each
1. A folder for each chunk with the same SAGs as the chunk files previously created. 


Once these files have been created then you can use ascp command line plugin to upload all of the folders to NCBI. The command will look something like this

#Folder with all of your fasta files
contigfolders=/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/split_files/chunk*

for folder in ${contigfolders[@]}
do

ascp -i /mnt/scgc/simon/microg2p/Data/GoM_uploads/aspera.openssh -QT -l100m -k1 -d ${folder} subasp@upload.ncbi.nlm.nih.gov:uploads/jacob.munsonmcgee_msu.montana.edu_bYXjnbY8/



In [15]:
# replace some of the data that was initially missing. It can be omitted from other uploads

sample_path = "/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/Biosample_attributes.txt"
sample=pd.read_csv(sample_path, sep='\t')    
sample[['*geo_loc_name', '*isolation_source']]=sample[['*geo_loc_name', '*isolation_source']].fillna(value='Northern Atlantic')
sample['*lat_lon']=sample['*lat_lon'].fillna('43.001_-11.333')
sample['*geo_loc_name']=sample['*geo_loc_name'].replace(['Northern Atlantic Ocean'], 'Atlantic Ocean')
sample['*geo_loc_name']=sample['*geo_loc_name'].replace(['Equitorial atlantic'], 'Atlantic Ocean')
sample['*geo_loc_name']=sample['*geo_loc_name'].replace(['Southern Atlantic'], 'Atlantic Ocean')
sample['*geo_loc_name']=sample['*geo_loc_name'].replace(['Juan De Fuca ridge'], 'Pacific Ocean: Juan De Fuca ridge')
sample.to_csv(sample_path, sep='\t')


array(['USA: East Boothbay ME', 'Atlantic Ocean',
       'Pacific Ocean: Juan De Fuca ridge'], dtype=object)

In [16]:
for i,chunk in enumerate(pd.read_csv(sample_path, chunksize=400, sep='\t')):
    chunk.to_csv('/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/split_files/biosample_attributes_chunk{}.txt'.format(i), index=False, sep='\t')
    


In [3]:

genome_path = "/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/SAG_genome_attributes.txt"
for i,chunk in enumerate(pd.read_csv(genome_path, chunksize=400, sep='\t')):
    chunk.to_csv('/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/split_files/SAG_genome_attributes_chunk{}.txt'.format(i), index=False, sep='\t')

  

There is probably a good argument to be made that I should do the chunk once on the biosample_attributes file and then use those chunk files to extract the SAGs that are in a chunk file from the genome_attributes file and create the first genome_attributes chunk file in a similar fashion to how I created the fasta chunk folders.
Oh well for now I am not going to modify the code but I will think about it for the future.

  

In [36]:
import glob
import shutil

# gather all of the SAG files
contigfiles=glob.glob('/mnt/scgc/simon/microg2p/Data/GoM_uploads/SAG_contigs/*.fasta')
len(contigfiles)

# gather all of the tables that were previously produced
tablefiles=glob.glob('/mnt/scgc/simon/microg2p/Data/GoM_uploads/NCBI_submission_files/split_files/SAG_genome_attributes_chunk*.txt')
len(tablefiles)

# remove all folders with SAG contigs yes this is brute force and makes it slower 
# but it also ensures that only the files within each table are in the folders
! rm -r *contigs

# for each table (chunk) do the following
#     1. extract the chunk # to keep it consistent with the table in case it is not read from 0-18
#     2. empty anything that is already in that folder with that chunk# and remake it
#     3. Then read the table and extract the SAGs that are part of it 
#     4. for every SAG check if the SAG name is in the fasta name
#         if it is copy it to the new folder and append it to the list
        
for file in tablefiles:
    path,chunk=file.rsplit('_', 1)
    chunk=chunk[:-4]
    
    folder=chunk+"_SAG_contigs"
    safe_make_dir(folder)
    
    df=pd.read_csv(file, sep='\t')
    names=df['filename'].to_list()
    cpfiles=[]
    for SAG in names:
        for i in contigfiles:
            if SAG in i:
                if i not in folder:
                    shutil.copy(i, folder)
                    cpfiles.append(i)
    print(len(glob.glob(folder+'/*')), "files were copied to", folder)
    


Folder created
400 files were copied to chunk8_SAG_contigs
Folder created
400 files were copied to chunk9_SAG_contigs
Folder created
400 files were copied to chunk0_SAG_contigs
Folder created
400 files were copied to chunk1_SAG_contigs
Folder created
400 files were copied to chunk2_SAG_contigs
Folder created
400 files were copied to chunk3_SAG_contigs
Folder created
400 files were copied to chunk4_SAG_contigs
Folder created
400 files were copied to chunk5_SAG_contigs
Folder created
400 files were copied to chunk6_SAG_contigs
Folder created
400 files were copied to chunk7_SAG_contigs
Folder created
400 files were copied to chunk10_SAG_contigs
Folder created
400 files were copied to chunk11_SAG_contigs
Folder created
400 files were copied to chunk12_SAG_contigs
Folder created
400 files were copied to chunk13_SAG_contigs
Folder created
400 files were copied to chunk14_SAG_contigs
Folder created
400 files were copied to chunk15_SAG_contigs
Folder created
400 files were copied to chunk16_SA

318